In [41]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [42]:
FEATURE_PATH = Path('/gpfs1/scratch/90days/s4436005/img_reg/features/Xception_features.csv')

In [43]:
features = pd.read_csv(FEATURE_PATH)

In [44]:
features

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,label
0,0,0.182635,0.531377,0.000000,0.0,0.105855,0.000000,0.000000,0.0,0.000000,...,0.018696,0.433531,0.0,0.000000,0.000000,0.0,0.0,0.721363,0.000000,cancer
1,1,0.000000,3.661574,0.000000,0.0,0.000000,0.000000,0.810756,0.0,1.167979,...,0.000000,1.918964,0.0,0.000000,0.000000,0.0,0.0,1.365252,0.000000,cancer
2,2,0.000000,5.968599,5.187304,0.0,0.000000,0.038286,0.630972,0.0,0.150833,...,0.025771,0.021439,0.0,0.000000,0.000000,0.0,0.0,10.631157,0.000000,non-cancer
3,3,0.000000,0.764106,14.787019,0.0,0.000000,1.182128,2.129080,0.0,4.889509,...,1.189364,0.000000,0.0,0.000000,0.000000,0.0,0.0,7.132992,0.000000,non-cancer
4,4,0.000000,3.808248,0.000000,0.0,0.000000,0.120199,0.289620,0.0,0.005790,...,0.001330,0.670733,0.0,0.000000,0.000000,0.0,0.0,3.813467,0.000000,cancer
5,5,0.000000,0.849750,1.656242,0.0,0.000000,0.326390,0.084029,0.0,0.533123,...,0.056564,0.119679,0.0,0.000000,0.000000,0.0,0.0,1.655943,0.000000,cancer
6,6,0.000000,1.067268,0.000000,0.0,0.000000,0.101069,0.153866,0.0,0.000000,...,0.061033,0.234052,0.0,0.000000,0.000000,0.0,0.0,0.227158,0.000000,non-cancer
7,7,0.000000,3.519799,0.000000,0.0,0.000000,0.000000,7.866794,0.0,2.199894,...,0.000000,0.000000,0.0,0.014934,0.000000,0.0,0.0,1.149581,0.000000,non-cancer
8,8,0.000000,34.331512,28.919876,0.0,0.000000,3.745589,2.264831,0.0,50.823680,...,0.514437,3.349199,0.0,0.000000,0.000000,0.0,0.0,9.946124,0.000000,non-cancer
9,9,0.000000,2.491828,0.346871,0.0,0.000000,0.033917,0.426171,0.0,0.000000,...,0.092640,0.810430,0.0,0.000000,0.000000,0.0,0.0,7.252862,0.000000,cancer


In [45]:
features = features.drop('Unnamed: 0', axis = 'columns')

In [46]:
features

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,label
0,0.182635,0.531377,0.000000,0.0,0.105855,0.000000,0.000000,0.0,0.000000,0.000000,...,0.018696,0.433531,0.0,0.000000,0.000000,0.0,0.0,0.721363,0.000000,cancer
1,0.000000,3.661574,0.000000,0.0,0.000000,0.000000,0.810756,0.0,1.167979,0.000000,...,0.000000,1.918964,0.0,0.000000,0.000000,0.0,0.0,1.365252,0.000000,cancer
2,0.000000,5.968599,5.187304,0.0,0.000000,0.038286,0.630972,0.0,0.150833,0.000000,...,0.025771,0.021439,0.0,0.000000,0.000000,0.0,0.0,10.631157,0.000000,non-cancer
3,0.000000,0.764106,14.787019,0.0,0.000000,1.182128,2.129080,0.0,4.889509,0.000000,...,1.189364,0.000000,0.0,0.000000,0.000000,0.0,0.0,7.132992,0.000000,non-cancer
4,0.000000,3.808248,0.000000,0.0,0.000000,0.120199,0.289620,0.0,0.005790,0.000000,...,0.001330,0.670733,0.0,0.000000,0.000000,0.0,0.0,3.813467,0.000000,cancer
5,0.000000,0.849750,1.656242,0.0,0.000000,0.326390,0.084029,0.0,0.533123,0.000000,...,0.056564,0.119679,0.0,0.000000,0.000000,0.0,0.0,1.655943,0.000000,cancer
6,0.000000,1.067268,0.000000,0.0,0.000000,0.101069,0.153866,0.0,0.000000,0.000000,...,0.061033,0.234052,0.0,0.000000,0.000000,0.0,0.0,0.227158,0.000000,non-cancer
7,0.000000,3.519799,0.000000,0.0,0.000000,0.000000,7.866794,0.0,2.199894,0.000000,...,0.000000,0.000000,0.0,0.014934,0.000000,0.0,0.0,1.149581,0.000000,non-cancer
8,0.000000,34.331512,28.919876,0.0,0.000000,3.745589,2.264831,0.0,50.823680,0.000000,...,0.514437,3.349199,0.0,0.000000,0.000000,0.0,0.0,9.946124,0.000000,non-cancer
9,0.000000,2.491828,0.346871,0.0,0.000000,0.033917,0.426171,0.0,0.000000,0.000000,...,0.092640,0.810430,0.0,0.000000,0.000000,0.0,0.0,7.252862,0.000000,cancer


In [47]:
y = features['label']

In [48]:
lb = LabelBinarizer()
Y = lb.fit_transform(y)

In [49]:
X = np.array(features.drop('label', axis = 'columns'))

In [50]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 42)

In [51]:
model = Sequential()
model.add(Dense(512, input_shape=(2048,), activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics=['accuracy'] )

In [52]:
H = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 100, batch_size = 32)

Train on 3840 samples, validate on 3840 samples
Epoch 1/100
3840/3840 [==============================] - 1s 335us/step - loss: 1.3600 - acc: 0.7576 - val_loss: 0.3627 - val_acc: 0.8573
Epoch 2/100
3840/3840 [==============================] - 1s 252us/step - loss: 0.4987 - acc: 0.8320 - val_loss: 0.3225 - val_acc: 0.8773
Epoch 3/100
3840/3840 [==============================] - 1s 245us/step - loss: 0.3906 - acc: 0.8479 - val_loss: 0.3308 - val_acc: 0.8721
Epoch 4/100
3840/3840 [==============================] - 1s 251us/step - loss: 0.3495 - acc: 0.8586 - val_loss: 0.3301 - val_acc: 0.8701
Epoch 5/100
3840/3840 [==============================] - 1s 245us/step - loss: 0.2964 - acc: 0.8758 - val_loss: 0.3179 - val_acc: 0.8719
Epoch 6/100
3840/3840 [==============================] - 1s 239us/step - loss: 0.2806 - acc: 0.8786 - val_loss: 0.3304 - val_acc: 0.8703
Epoch 7/100
3840/3840 [==============================] - 1s 252us/step - loss: 0.2707 - acc: 0.8898 - val_loss: 0.3206 - val_acc: 

Epoch 60/100
3840/3840 [==============================] - 1s 211us/step - loss: 0.0459 - acc: 0.9805 - val_loss: 0.7866 - val_acc: 0.8677
Epoch 61/100
3840/3840 [==============================] - 1s 241us/step - loss: 0.0499 - acc: 0.9792 - val_loss: 0.6920 - val_acc: 0.8695
Epoch 62/100
3840/3840 [==============================] - 1s 252us/step - loss: 0.0352 - acc: 0.9883 - val_loss: 0.7385 - val_acc: 0.8755
Epoch 63/100
3840/3840 [==============================] - 1s 247us/step - loss: 0.0334 - acc: 0.9870 - val_loss: 0.7542 - val_acc: 0.8755
Epoch 64/100
3840/3840 [==============================] - 1s 234us/step - loss: 0.0532 - acc: 0.9802 - val_loss: 0.7507 - val_acc: 0.8732
Epoch 65/100
3840/3840 [==============================] - 1s 220us/step - loss: 0.0643 - acc: 0.9781 - val_loss: 0.6814 - val_acc: 0.8591
Epoch 66/100
3840/3840 [==============================] - 1s 220us/step - loss: 0.0740 - acc: 0.9750 - val_loss: 0.6341 - val_acc: 0.8708
Epoch 67/100
3840/3840 [==========